# [Data Preparation]


# 1. Short ideas data from Seeking Alpha

Seeking Alpha short selling article data was collected by web scraping, using Scrapy. The scraper ran on an Amazon Web Services (AWS) EC2 instance, and the results were stored in CSV format in an S3 bucket. There was significant web scraping issue because there is a strict limitation on the amount of data that can be scraped at once from SA. This was resolved by using Amazon EC2 and tweaking user-agent setting on Scrapy. 

Seeking Alpha web pages are pretty well structured therefore the following fields can be obtained after a little bit of processing.
* article id, publication date, editors' pick info, url
* author id, name and url
* company name and ticker

Scrapy codes for the web scraping can be found here and the data processing codes are shown below.


### Import libraries

In [1]:
# Import libraries for data construction
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# To plot figures on jupyter
%matplotlib inline

# Set max rows and columns to display
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)

### Concatenate web scraped csv files
Short ideas data is scraped and save in many csv files. Let's concatenate them and create a dataframe.

In [2]:
# Concatenate web-scraped csv files 
import glob, os

path = '/Users/Woosik/capstone_my/data/list'
all_files = glob.glob(os.path.join(path, '*.csv'))

df_from_each_file = (pd.read_csv(f, encoding = 'ISO-8859-1') for f in all_files)
df = pd.concat(df_from_each_file, ignore_index=True)

### Clean the dataframe and create variables

In [3]:
# Clean variable name and drop duplicates
df.rename(columns={'info':'article_info'}, inplace=True)
df.drop_duplicates(inplace=True)

# Some articles have no ticker information. Usually those articles discuss overall sector or market.
# Let's ignore (delete) those articles for this project.
df.dropna(axis=0, inplace=True)

# Clean author_name column
df['author_name'] = df.apply(lambda x: x.author_name.replace(x.ticker+',', ''), axis=1)

# Extract datetime from article_info column
import re
from datetime import datetime
df['article_info'] = [i.replace('May', 'May.') for i in df.article_info]
df['date_temp'] = [re.split('\.|M,', i)[0][-3:] + '.' + re.split('\.|M,', i)[1] + 'M' for i in df.article_info]
df['date'] = pd.to_datetime(df['date_temp'], format='%b. %d, %Y, %I:%M %p') 

# Extract Editors' pick info as boolean
df['editors_pick'] = df['article_info'].str.split(',').str[0] == "Editors' Pick"

# Create a variable indicating the number of stocks discussed in the article
df['num_ticker'] = df.ticker.str.count(',') + 1

# Create full url for later scraping
df['author_url'] = 'https://seekingalpha.com' + df['author_link']
df['article_url'] = 'https://seekingalpha.com' + df['article_link']

# Sort dataframe and reset index
df.sort_values(by=['date'], inplace=True)
df.reset_index(drop=True, inplace=True)

### Number of stocks discussed in the article

SA assigns a unique id to each article. In addition, SA editors tag each article with one or more stock tickers prior to publication. Single-ticker articles focus solely on one stock, making it relatively easy to extract the author’s opinion on that company. Multiple-ticker articles discuss more than one stock in the same article, rendering extraction of the author’s various opinions for each of the tagged stocks difficult, if not impossible. I therefore focus my analysis on single-ticker articles.

In [4]:
df.num_ticker.value_counts()

1    14003
2      350
3      253
Name: num_ticker, dtype: int64

In [5]:
# Let's keep single-ticker articles only for this project.
df = df[df['num_ticker'] == 1]
df.reset_index(drop=True, inplace=True)

### Keep relevant columns only

In [6]:
df = df[['date','editors_pick','article_id','author_id','author_name',
         'company_name','ticker','num_ticker','article_link','author_link',
         'article_url','author_url','article_info']]

In [7]:
df.to_pickle('data/df_single_ticker_articles.pkl')

# 2. SA opinion leaders data from Wayback Machine

The list of opinion leaders of short ideas for every quarters during sample time period is manually collected from Wayback Machine (http://web.archive.org) and saved in excel format.

In [8]:
# Opinion leader data from Wayback Machine
dfl = pd.read_excel('data/opinion_leader_list.xlsx')

In [9]:
dfl.head()

,rank,leader_2012Q1,leader_2012Q2,leader_2012Q3,leader_2012Q4,leader_2013Q1,leader_2013Q2,leader_2013Q3,leader_2013Q4,leader_2014Q1,leader_2014Q2,leader_2014Q3,leader_2014Q4,leader_2015Q1,leader_2015Q2,leader_2015Q3,leader_2015Q4,leader_2016Q1,leader_2016Q2,leader_2016Q3,leader_2016Q4
0,1,Paulo Santos,Paulo Santos,Bidness Etc,Bidness Etc,Ashraf Eassa,Core Equity Research,Kofi Bofah,Quoth the Raven,Quoth the Raven,Quoth the Raven,Quoth the Raven,Paulo Santos,Anton Wahlman,Shock Exchange,Paulo Santos,Paulo Santos,Anton Wahlman,Montana Skeptic,Paulo Santos,Paulo Santos
1,2,Bill Maurer,David White,Bill Maurer,Bill Maurer,Bill Maurer,Ashraf Eassa,Ashraf Eassa,Bill Maurer,Michael Blair,Michael Blair,Michael Blair,Michael Blair,Paulo Santos,Paulo Santos,Anton Wahlman,Shock Exchange,Montana Skeptic,Paulo Santos,Montana Skeptic,Montana Skeptic
2,3,Rocco Pendola,Gutone,Saibus Research,Saibus Research,Paulo Santos,Markos Kaminis,Michael Blair,Michael Blair,Bill Maurer,Don Dion,Shock Exchange,Anton Wahlman,Michael Blair,Anton Wahlman,Shock Exchange,Josh Arnold,Paulo Santos,Bill Maurer,Shock Exchange,Bill Maurer
3,4,Shmulik Karpf,Bill Maurer,Shane Blackmon,Peter Pham,Dana Blankenhorn,Bill Maurer,Bill Maurer,Don Dion,EnhydrisPECorp,The Pump Stopper,Paulo Santos,The Specialist,Brian Nichols,Whitney Tilson,Short/Long Trader,Brandon Dempster,Michael Blair,Anton Wahlman,Anton Wahlman,Shock Exchange
4,5,Takeover Analyst,Richard Saintvilus,Paulo Santos,Modernist,David White,Paulo Santos,Richard Pearson,Seeking Profits,Seeking Profits,Bill Maurer,Don Dion,Shock Exchange,Matt Stewart,Logical Thought,Orange Peel Investments,Anton Wahlman,Shock Exchange,Mark Hibben,Orange Peel Investments,Orange Peel Investments


# 3. Daily stock data from CRSP

### Create empty columns for the stock related variables in the short ideas dataframe

In [10]:
# Open the short ideas dataframe
df = pd.read_pickle('data/df_single_ticker_articles.pkl')

# Create empty columns
df['5d_ret'] = np.nan  # 5 days (1 week)
df['10d_ret'] = np.nan  # 10 days (2 weeks)
df['20d_ret'] = np.nan  # 20 days (1 month)
df['40d_ret'] = np.nan  # 40 days (2 months)
df['60d_ret'] = np.nan  # 60 days (3 months)
df['80d_ret'] = np.nan  # 80 days (4 months)
df['100d_ret'] = np.nan  # 100 days (5 months)
df['mkt_cap'] = np.nan
df['siccd'] = np.nan

### Create start and end date variables to slice the dataset from CRSP and Yahoo Finance

In [11]:
# Create start and end date variables to slice CRSP dataset
from datetime import timedelta
df['start_date'] = df['date'].dt.date
df['end_date'] = df['start_date'] + timedelta(days=150)  # little bit longer than 5 months

# Convert dates to string so that fix_yahoo_finance can recognize them
def date_to_string(x):
    try:
        return datetime.strftime(x, '%Y-%m-%d')
    except:
        pass
df['start_date'] = df['start_date'].apply(date_to_string)
df['end_date'] = df['end_date'].apply(date_to_string)

### Write functions that extract necessary information from CRSP daily stock dataset

Daily stock data including ticker, return, price, number of shares outstanding and sic codes is obtained from CRSP through wrds (http://wrds-web.wharton.upenn.edu) and saved in seperate csv file by year.


In [12]:
def df_generator(csv_file_name):
    
    """Read CRSP stock daily csv file (already downloaded), clean and return dataframe.

    Args:
        csv_file_name: string of csv file name including '.csv'  ex) 'crsp_2012.csv'
    Returns:
        dataframe containing stock info such as date, ticker, return, mkt_cap and sic code. 
        
    """
    
    # Create dictionary that contains dataframe for each year
    df = pd.read_csv('data/CRSP/'+csv_file_name)
    df.drop_duplicates(inplace=True)
    df.columns = df.columns.str.lower()
    df.dropna(subset=['ticker'], inplace=True)
    
    # Change type of date, from string to datetime
    df['date'] = pd.to_datetime(df['date'].astype(str), format='%Y%m%d')
    
    # Clean price data following the variable instrction from wrds
    df['prc'] = [prc*-1 if prc < 0 else prc for prc in df['prc']]
    df['prc'] = [np.nan if prc == 0 else prc for prc in df['prc']]
    
    # Calculate stock market cap (stock size)
    df['mkt_cap'] = df.prc * df.shrout
    
    # Keep necessary variables only
    df = df[['date', 'ticker', 'ret', 'mkt_cap', 'siccd']]
    
    # Reset index
    df.reset_index(drop=True, inplace=True)
    
    return df

In [13]:
def quote_generator(df, ticker, start_date, end_date):
    
    """Slice dataframe for specified ticker and time period.

    Args:
        df: dataframe with stock info
        ticker: ticker of stock
        start_date: start date of quote
        end_date: end_date of quote
    Returns:
        quote containing stock info for specified ticker and time period. 
        
    """
    return df[(df['ticker']==ticker) & ((df['date'] > start_date) & (df['date'] <= end_date))]    

### Fill in the empty cells of stock related variables in the short ideas dataframe
In this project, the holding period returns from 5 business days (1 week) to 100 business days (5 months) will be used for the short portfolio performance analysis. This is because, due to the stock borrowing fee, it is hard to imagine that short sellers hold their short position longer than 5 months.

In [14]:
# Calculate compound returns of the stocks recommended in short ideas articles in year 2012~2016 (index: 0 to 12338)
for i in range(12339):
    yr = df.iloc[[i]].date.dt.year.astype(str)[i]
    print('currently on line: ' + str(i))
    try:
        quotes = quote_generator(df_generator('crsp_'+yr+'.csv'), df.ticker[i], df.start_date[i], df.end_date[i])
        quotes.reset_index(drop=True, inplace=True)
        quotes['cum_ret'] = (1 + quotes.ret.astype(float)).cumprod() - 1
        df['5d_ret'][i] = quotes.iloc[5,5]  # 5 days (1 week)
        df['10d_ret'][i] = quotes.iloc[10,5]  # 10 days (2 weeks)
        df['20d_ret'][i] = quotes.iloc[20,5]  # 20 days (1 month)
        df['40d_ret'][i] = quotes.iloc[40,5]  # 40 days (2 months)
        df['60d_ret'][i] = quotes.iloc[60,5]  # 60 days (3 months)
        df['80d_ret'][i] = quotes.iloc[80,5]  # 80 days (4 months)
        df['100d_ret'][i] = quotes.iloc[100,5]  # 100 days (5 months)
        df['mkt_cap'][i] = quotes.iloc[0,3]  # adjusted prc at start date (or the closest business day)
        df['siccd'][i] = quotes.iloc[0,4]
    except:
        continue

# Save
df.to_pickle('data/df_yahoo_crsp.pkl')

### Clean and create variables

In [15]:
df = pd.read_pickle('data/df_yahoo_crsp.pkl')

In [16]:
df.head()

,date,editors_pick,article_id,author_id,author_name,company_name,ticker,num_ticker,article_link,author_link,article_url,author_url,article_info,start_date,end_date,start_prc,5d_prc,10d_prc,20d_prc,40d_prc,60d_prc,80d_prc,100d_prc,5d_ret,10d_ret,20d_ret,40d_ret,60d_ret,80d_ret,100d_ret,mkt_cap,siccd
0,2012-01-01 11:18:00,False,316904,973944,Honne Capital,Mattress Firm Holding Corp.,MFRM,1,/article/316904-mattress-firm-should-provide-m...,/author/honne-capital/articles,https://seekingalpha.com/article/316904-mattre...,https://seekingalpha.com/author/honne-capital/...,"Û¢,Jan. 1, 2012, 11:18 AM,Û¢,Û¢,11åÊComments",2012-01-01,2012-05-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.034929,0.168177,0.374773,0.465286,0.643379,0.726604,0.670544,835782.75,9999
1,2012-01-02 05:56:00,False,316946,360252,The GeoTeam,Raystream Inc.,RAYS,1,/article/316946-raystream-remains-under-scruti...,/author/the-geoteam/articles,https://seekingalpha.com/article/316946-raystr...,https://seekingalpha.com/author/the-geoteam/ar...,"Û¢,Jan. 2, 2012, 5:56 AM,Û¢,Û¢,26åÊComments",2012-01-02,2012-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012-01-03 09:00:00,False,317052,201452,Gary Weiss,"Overstock.com, Inc.",OSTK,1,/article/317052-does-patrick-byrnes-overstock-...,/author/gary-weiss/articles,https://seekingalpha.com/article/317052-does-p...,https://seekingalpha.com/author/gary-weiss/art...,"Û¢,Jan. 3, 2012, 9:00 AM,Û¢",2012-01-03,2012-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.089493,-0.115434,-0.105058,-0.207523,-0.320363,-0.217901,-0.141379,174825.29,5999
3,2012-01-04 14:40:00,False,317392,1072567,Ryan Canady,Lululemon Athletica Inc.,LULU,1,/article/317392-lululemon-is-a-sell-and-the-in...,/author/ryan-canady/articles,https://seekingalpha.com/article/317392-lulule...,https://seekingalpha.com/author/ryan-canady/ar...,"Û¢,Jan. 4, 2012, 2:40 PM,Û¢,Û¢,80åÊComments",2012-01-04,2012-06-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.197263,0.175366,0.262560,0.345453,0.461190,0.486801,0.426781,5668688.40,5600
4,2012-01-04 20:01:00,False,317485,1017993,Bill Maurer,"Netflix, Inc.",NFLX,1,/article/317485-is-it-possible-that-we-are-too...,/author/bill-maurer/articles,https://seekingalpha.com/article/317485-is-it-...,https://seekingalpha.com/author/bill-maurer/ar...,"Û¢,Jan. 4, 2012, 8:01 PM,Û¢,Û¢,49åÊComments",2012-01-04,2012-06-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.145431,0.245991,0.571534,0.377378,0.416656,0.011312,-0.162958,4393140.70,7841


In [17]:
# Drop unnecessary variables
df.drop(['start_prc','5d_prc','10d_prc','20d_prc','40d_prc','60d_prc','80d_prc','100d_prc'], axis=1, inplace=True)
df.drop(df[df.date.dt.year==2017].index, inplace=True)
df.dropna(inplace=True)

# Create date variables for later analysis
df.rename(columns={'date': 'datetime'}, inplace=True)
df['date'] = df['datetime'].dt.date
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['datetime'].dt.year
df['yr_qt'] = df['datetime'].dt.to_period("Q")
df['yr_qt_str'] = df.yr_qt.astype(str)

# Manually drop rows containing erroneous return values
df.drop(df[(df['5d_ret']>10)|(df['40d_ret']>9)].index, inplace=True)

# Clean sic code
df.loc[df.siccd=='Z', 'siccd'] = 0
df.siccd = df.siccd.astype(int)
df.siccd = [format(sic, '04d') for sic in df.siccd]
df['sic2'] = df['siccd'].str[:2].astype(int)

# Assign sic major industry classification
df.loc[(df.sic2>=1)&(df.sic2<=9), 'sic2_name'] = 'Agriculture'
df.loc[(df.sic2>=10)&(df.sic2<=14), 'sic2_name'] = 'Mining'
df.loc[(df.sic2>=15)&(df.sic2<=17), 'sic2_name'] = 'Construction'
df.loc[(df.sic2>=20)&(df.sic2<=39), 'sic2_name'] = 'Manufacturing'
df.loc[(df.sic2>=40)&(df.sic2<=49), 'sic2_name'] = 'Transportation'
df.loc[(df.sic2>=50)&(df.sic2<=51), 'sic2_name'] = 'Wholesale'
df.loc[(df.sic2>=52)&(df.sic2<=59), 'sic2_name'] = 'Retail'
df.loc[(df.sic2>=60)&(df.sic2<=67), 'sic2_name'] = 'Finance'
df.loc[(df.sic2>=70)&(df.sic2<=89), 'sic2_name'] = 'Services'
df.loc[(df.sic2>=90)&(df.sic2<=98), 'sic2_name'] = 'Public Admin'
df.loc[df.sic2==99, 'sic2_name'] = 'Nonclassifiable'
df.loc[df.sic2==0, 'sic2_name'] = 'N/A'

# Change sic industry category for later analysis
df.loc[df.sic2_name == 'Transportation', 'sic2_name'] = 'Telecom'
df.loc[df.company_name == 'Tesla, Inc.', 'sic2_name'] = 'Manufacturing'
df.loc[df.company_name == 'Facebook', 'sic2_name'] = 'Services'
df.loc[df.company_name == 'Groupon, Inc.', 'sic2_name'] = 'Services'
df.loc[df.company_name == 'GoPro', 'sic2_name'] = 'Manufacturing'
df.loc[df.company_name == 'ANGI Homeservices Inc.', 'sic2_name'] = 'Services'
df.loc[df.company_name == 'Zynga', 'sic2_name'] = 'Services'
df.loc[df.company_name == 'Zillow Group, Inc.', 'sic2_name'] = 'Services'
df.loc[df.ticker == 'OMEX', 'sic2_name'] = 'Mining'

df.loc[(df.sic2_name!='Manufacturing')&(df.sic2_name!='Services')&(df.sic2_name!='Retail')&
       (df.sic2_name!='Telecom')&(df.sic2_name!='Finance')&(df.sic2_name!='Mining')&
       (df.sic2_name!='Wholesale')&(df.sic2_name!='Nonclassifiable'), 'sic2_name'] = 'Etc.'

# Reset index
df.reset_index(drop=True, inplace=True)

# 4. Merge the dataframe with disclosure information 

In [18]:
# Load the dataframe containing the disclosure information and article body
df_nlp = pd.read_pickle('data/df_all_VADER.pkl')

In [19]:
# Construct categorical variable about disclosure 
# 1. short/ 2. long/ 3. no position- no plan/ 4. no position- may initiate a short position/ 5. no info)
df_nlp['body_str'] = df_nlp['body'].apply(lambda x: ' '.join(map(str, x)))
conditions = [
    (df_nlp.disclosure.notnull()&df_nlp.disclosure.str.contains('are short|am short|is short')) 
    | (df_nlp.disclosure.isnull()&df_nlp.body_str.str.contains('are short|am short|is short')), # 1. short
    (df_nlp.disclosure.notnull()&df_nlp.disclosure.str.contains('are long|am long|is long')) 
    | (df_nlp.disclosure.isnull()&df_nlp.body_str.str.contains('are long|am long|is long')), # 2. long
    (df_nlp.disclosure.notnull()&df_nlp.disclosure.str.contains('no positions in any stocks mentioned, and no plans')) 
    | (df_nlp.disclosure.isnull()&df_nlp.body_str.str.contains('no positions in any stocks mentioned, and no plans')), # 3. no position- no plan
    (df_nlp.disclosure.notnull()&df_nlp.disclosure.str.contains('no positions in any stocks mentioned, but may initiate')) 
    | (df_nlp.disclosure.isnull()&df_nlp.body_str.str.contains('no positions in any stocks mentioned, but may initiate'))] # 4. no position- may initiate
choices = ['Short', 'Long', 'No Position No Plan', 'No Position But May']
df_nlp['disclosure_cat'] = np.select(conditions, choices, default='n/a')
df_nlp.drop('body_str', axis=1, inplace=True) 

# Merge df with df_nlp
df_inner = df.merge(df_nlp, how='inner', on='article_id')

# Keep necessary variables only
df_inner = df_inner[['datetime', 'date_x', 'article_id', 'author_id_x', 'author_name_x', 
                     'editors_pick_x', 'company_name_x', 'ticker_x', 'article_url_x', 'author_url_x', 
                     '5d_ret', '10d_ret', '20d_ret', '40d_ret', '60d_ret', '80d_ret', '100d_ret',
                     'mkt_cap', 'siccd', 'sic2', 'sic2_name', 'year', 'yr_qt', 'yr_qt_str',
                     'author_bio', 'body', 'disclosure_cat', 'Num_syl', 'Num_words', 'Num_sent', 
                     'neg_ratio_abs', 'pos_ratio_abs', 'unc_ratio_abs', 'v_neg', 'v_pos', 'v_neu', 'v_comp']]

# Change variables name
df_inner.rename(columns={'date_x':'date', 'author_id_x':'author_id', 'author_name_x':'author_name',
                         'editors_pick_x':'editors_pick', 'company_name_x':'company_name',
                         'ticker_x':'ticker', 'article_url_x': 'article_url', 
                         'author_url_x':'author_url'}, inplace=True)

# Keep data from 2012 to 2015
df_inner = df_inner[df_inner.year!=2016]

# Save
df_inner.to_pickle('data/df_inner_nlp.pkl')

In [20]:
df_inner.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7873 entries, 0 to 7872
Data columns (total 37 columns):
datetime          7873 non-null datetime64[ns]
date              7873 non-null datetime64[ns]
article_id        7873 non-null int64
author_id         7873 non-null int64
author_name       7873 non-null object
editors_pick      7873 non-null bool
company_name      7873 non-null object
ticker            7873 non-null object
article_url       7873 non-null object
author_url        7873 non-null object
5d_ret            7873 non-null float64
10d_ret           7873 non-null float64
20d_ret           7873 non-null float64
40d_ret           7873 non-null float64
60d_ret           7873 non-null float64
80d_ret           7873 non-null float64
100d_ret          7873 non-null float64
mkt_cap           7873 non-null float64
siccd             7873 non-null object
sic2              7873 non-null int64
sic2_name         7873 non-null object
year              7873 non-null int64
yr_qt            